In [1]:
import os

In [2]:
%pwd

'd:\\My\\AboutData\\Project\\Weather-Prediction\\notebooks'

In [3]:
os.chdir("../")
%pwd

'd:\\My\\AboutData\\Project\\Weather-Prediction'

In [4]:
from dataclasses import dataclass
from pathlib import Path

In [5]:
@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    model_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [6]:
from src.constants import *
from src.utils.helper import read_yaml, create_directories, save_json

In [7]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILEPATH, params_filepath = PARAMS_FILEPATH, schema_filepath = SCHEMA_FILEPATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.RandomForestClassifier
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            test_data_path = config.test_data_path,
            model_path = config.model_path,
            model_params = params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri = "https://dagshub.com/jayawinata100/Weather-Prediction.mlflow"
        )

        return model_evaluation_config

In [8]:
import os
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from urllib.parse import urlparse
import mlflow
import numpy as np
import joblib
import dagshub

In [9]:
dagshub.init(repo_owner='jayawinata100', repo_name='Weather-Prediction', mlflow=True)

[2025-05-23 15:13:51,552: INFO: tokens: Removed expired tokens from the token cache]


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

d:\My\AboutData\Project\Weather-Prediction\weather-prediction-venv\lib\site-packages\rich\live.py:231: UserWarning:
install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=f8634b43-7b5e-41e3-b948-959478d32c25&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=c28242d7c0e1502281f5909e22936773c2e8515df3e358896c3f9ea9b881bb2e


[2025-05-23 15:13:57,519: INFO: _client: HTTP Request: POST https://dagshub.com/login/oauth/middleman "HTTP/1.1 200 OK"]


[2025-05-23 15:13:58,156: INFO: _client: HTTP Request: POST https://dagshub.com/login/oauth/access_token "HTTP/1.1 200 OK"]
[2025-05-23 15:13:58,721: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


...

[2025-05-23 15:13:58,726: INFO: helpers: Accessing as jayawinata100]
[2025-05-23 15:13:59,256: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/jayawinata100/Weather-Prediction "HTTP/1.1 200 OK"]
[2025-05-23 15:13:59,980: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "jayawinata100/Weather-Prediction"

[2025-05-23 15:13:59,985: INFO: helpers: Initialized MLflow to track repo "jayawinata100/Weather-Prediction"]


Repository jayawinata100/Weather-Prediction initialized!

[2025-05-23 15:13:59,988: INFO: helpers: Repository jayawinata100/Weather-Prediction initialized!]


In [12]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, predicted):
        accuracy = accuracy_score(actual, predicted)
        precision = precision_score(actual, predicted, average='weighted')
        recall = recall_score(actual, predicted, average='weighted')
        f1 = f1_score(actual, predicted, average='weighted')
        return accuracy, precision, recall, f1

    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_value = model.predict(test_x)
            accuracy, precision, recall, f1 = self.eval_metrics(test_y, predicted_value)

            score = {'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1_score': f1}
            save_json(path=Path(self.config.metric_file_name), data=score)

            mlflow.log_params(self.config.model_params)

            mlflow.log_metric("accuracy", accuracy)
            # mlflow.log_text(str(classification_report(test_y, predicted_value)))

            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="RFClassifierModel")
            else:
                mlflow.sklearn.log_model(model, "model")

In [13]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2025-05-23 15:15:08,470: INFO: helper: yaml file: config\config.yaml loaded successfully]
[2025-05-23 15:15:08,472: INFO: helper: yaml file: params.yaml loaded successfully]
[2025-05-23 15:15:08,476: INFO: helper: yaml file: schema.yaml loaded successfully]
[2025-05-23 15:15:08,476: INFO: helper: created directory at: artifacts]
[2025-05-23 15:15:08,481: INFO: helper: created directory at: artifacts/model_evaluation]
[2025-05-23 15:15:09,026: INFO: helper: json file saved at: artifacts\model_evaluation\metrics.json]


d:\My\AboutData\Project\Weather-Prediction\weather-prediction-venv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
2025/05/23 15:15:29 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'RFClassifierModel'.
2025/05/23 15:15:34 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: RFClassifierModel, version 1
Created version '1' of model 'RFClassifierModel'.


🏃 View run sneaky-robin-355 at: https://dagshub.com/jayawinata100/Weather-Prediction.mlflow/#/experiments/0/runs/8169773f44284c188a682149e55b6b91
🧪 View experiment at: https://dagshub.com/jayawinata100/Weather-Prediction.mlflow/#/experiments/0
